In [1]:
import pandas as pd
from collections import Counter

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [87]:
inputfile = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_data_corpus.csv"
inputdict = "/home/francisco/GitHub/DQN-Event-Summarization/data/cnn_tokenized/cnn_total_corpus_smry.csv"

In [3]:
torch.manual_seed(1)

In [15]:
qdf = pd.read_csv(inputfile)
qdict = pd.read_csv(inputdict)

/home/francisco/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
query = qdf['query_id']
true_summaries = qdf['tstokens']
sentences = qdf[[x for x in qdf.columns if 'stokens_' in x]]

In [81]:
corpus = Counter()
for word in ['a' ,'b', 'c', 'a', 'd']:
    corpus[word] += 1

print(corpus)

Counter({'a': 2, 'b': 1, 'c': 1, 'd': 1})


In [78]:
def rougueScores(genSummary, refSummary):
    genTotal, refTotal, intersection = 0, 0, 0
    for token in list(set(list(refSummary.keys()) + list(genSummary.keys()) )):
        intersection += min(refSummary[token], genSummary[token])
        refTotal += refSummary[token]
        genTotal += genSummary[token]

    recall = intersection / float(refTotal) if float(refTotal) > 0 else 0
    prec   = intersection / float(genTotal) if float(genTotal) > 0 else 0
    f1 = (2 * recall * prec) / (recall + prec) if (recall + prec) > 0 else 0
    
    return recall, prec, f1

In [79]:
trueSummary = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])

predSummary0 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3])

predSummary1 = Counter([1, 1, 1, 2, 2, 2, 2, 3, 3, 4])

predSummary2 = Counter([1, 1, 1, 2, 2, 2, 2, 3])

predSummary3 = Counter([4])

In [80]:
print(rougueScores(predSummary0, trueSummary))
print(rougueScores(predSummary1, trueSummary))
print(rougueScores(predSummary2, trueSummary))
print(rougueScores(predSummary3, trueSummary))

(1.0, 1.0, 1.0)
(1.0, 0.9, 0.9473684210526316)
(0.8888888888888888, 1.0, 0.9411764705882353)
(0.0, 0.0, 0)


In [93]:
qdict.head()

,id,token,frequency,cumfreq,percent,cumpercent
0,40,by,4528679,4528679,0.007296,0.007296
1,17,try,4088095,8616774,0.006586,0.013882
2,9,once,3899082,12515856,0.006282,0.020163
3,114,case,3867709,16383565,0.006231,0.026394
4,3,45,3694710,20078275,0.005952,0.032347


In [109]:
corpus_dict = dict(zip(qdict['id'].values, qdict['token'].values))

In [126]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for token in sentence:
        vec[token] += 1
    return vec.view(1, -1)

In [129]:
class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, num_labels, vocab_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is num_labels!
        
        self.linear = nn.Linear(vocab_size, num_labels)

        # NOTE! The non-linearity log softmax does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec))


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [155]:
train_data = [
    (make_bow_vector([int(s) for s in sentences['stokens_0'][0].split(" ")], corpus_dict),
    torch.LongTensor([0]))
]

In [141]:
nlabels = 2 
vocab_size = len(corpus_dict)

In [142]:
model = BoWClassifier(nlabels, vocab_size)

for param in model.parameters():
    print(param)

log_probs = model(autograd.Variable(train_data[0]))
print(log_probs)

Parameter containing:
-2.8658e-03 -1.6441e-03  2.5561e-03  ...   4.4683e-04 -9.2455e-04  8.7829e-04
-2.1095e-03  2.1107e-03  2.8656e-03  ...   2.2872e-03  8.5265e-04  8.3847e-04
[torch.FloatTensor of size 2x111212]

Parameter containing:
1.00000e-03 *
  1.2245
 -0.9217
[torch.FloatTensor of size 2]

Variable containing:
-0.6966 -0.6897
[torch.FloatTensor of size 1x2]



In [151]:
test_data =  [
    (make_bow_vector([int(s) for s in sentences['stokens_0'][1].split(" ")], corpus_dict),
    torch.LongTensor([1]) )
    ]

In [158]:
for instance, label in test_data:
    log_probs = model(autograd.Variable(instance))
    print(log_probs)

Variable containing:
-0.6905 -0.6958
[torch.FloatTensor of size 1x2]



In [159]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [163]:
# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in train_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(instance)
        target = autograd.Variable(label)

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [165]:
for instance, label in test_data:
    bow_vec = autograd.Variable(instance)
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.4435 -1.0266
[torch.FloatTensor of size 1x2]

